In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.model_selection import train_test_split, cross_validate
from sklearn import preprocessing
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from lightgbm import LGBMRegressor

In [ ]:
df = pd.read_csv(f'../../../data/IFS/features_K_LGBM.csv')
K = df['K_VRH']
del df['material_id']
del df['structure']
del df['K_VRH']
df

In [ ]:
def prepro(df, target, scaling=True, test=0.2, seed=2021):
    
    x = df.to_numpy()
    y = target.to_numpy()
    
    if scaling:
        x = preprocessing.StandardScaler().fit_transform(x)
        
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test, shuffle=True, random_state=seed)
    
    return x_train, x_test, y_train, y_test

In [ ]:
def predict(model, x_train, x_test, y_train, y_test):
    
    elif model == 'LGBM':
        reg = LGBMRegressor(boosting_type='gbdt',
                            class_weight=None,
                            colsample_bytree=1.0,
                            importance_type='split',
                            learning_rate=0.05,
                            max_depth=-1,
                            min_child_samples=76,
                            min_child_weight=0.001,
                            min_split_gain=0.4,
                            n_estimators=190,
                            n_jobs=-1,
                            num_leaves=100,
                            objective=None,
                            random_state=4463,
                            reg_alpha=0.4,
                            reg_lambda=2,
                            silent='warn',
                            subsample=1.0,
                            subsample_for_bin=200000,
                            subsample_freq=0,
                            feature_fraction=0.8,
                            bagging_freq=1,
                            bagging_fraction=0.9)
    
    reg.fit(x_train, y_train)
    y_pred = reg.predict(x_test)
    
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    
    feat = reg.feature_importances_
    
    return r2, mse, mae, y_test, y_pred, feat

In [ ]:
def plot(y_test, y_pred):
    plt.figure(figsize=(4, 4))
    plt.scatter(y_test, y_pred, c='blue', alpha=0.4)
    plt.plot([0, 500], [0, 500], c='black', alpha=0.5)
    plt.xlabel('True')
    plt.ylabel('Pred')
    plt.show()

In [ ]:
model_list = ['LGBM']

In [ ]:
G_model_list = []
G_r2_list = []
G_mse_list = []
G_mae_list = []
G_rand_list = []

for i in tqdm(range(len(model_list))):
    model = model_list[i]
    print(model)
    for r in tqdm(range(0, 1000, 100)):
        print(r)
        x_train, x_test, y_train, y_test = prepro(df, K, scaling=True, test=0.2, seed=r)
        r2, mse, mae, y_test, y_pred, feat = predict(model, x_train, x_test, y_train, y_test)
        print('R2 :', round(r2, 2))
        print('MSE :', round(mse, 2))
        print('MAE :', round(mae, 2))
        plot(y_test, y_pred)
        G_model_list.append(model)
        G_r2_list.append(round(r2, 4))
        G_mse_list.append(round(mse, 4))
        G_mae_list.append(round(mae, 4))
        G_rand_list.append(r)

In [ ]:
G_result_df = pd.DataFrame({'Model':G_model_list, 'R2':G_r2_list, 'MSE':G_mse_list, 'MAE':G_mae_list, 'Random State':G_rand_list})
G_result_df

In [ ]:
G_result_df.to_csv(f'./K_{model}_results.csv', index=False)